<a href="https://colab.research.google.com/github/Fathanhelmi/Twitter-scraping/blob/main/Crawling_Twitter_Kelompok_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tweepy


In [2]:
import tweepy
import pandas as pd
# import gspread
import os

In [3]:

import tweepy
import pandas as pd
# import gspread
import os


# Replace with your actual credentials
consumer_key = "d4m9UVDY29JfNxSBkgRgtVV8k"
consumer_secret = "WOVDKgDGMj4POMrAv6TRPXlkD4gtXVlT67T99pI78qSrQcZJQ1"
access_token = "1990408864047697920-oFWUiSHciYzi5Y7l7QgAf9l1daWwS8"
access_token_secret = "T075M1Zj4hyMoSkLuu8FVjeQYbIWDKaI36OKOTBqI95BA"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAP%2Fp5QEAAAAAiE6%2B4D2dPEK5qZ5wNm5CZi9WCDY%3DrmoD0xEu1oJBu4x6w8Bx4KdEBlaPEmharcVc7m4iStSkbDaQYQ"

try:
    # Authenticate with the Twitter API
    auth = tweepy.OAuth1UserHandler(
        consumer_key,
        consumer_secret,
        access_token,
        access_token_secret,
        bearer_token
    )
    api = tweepy.API(auth)

    # Verify credentials (optional but recommended)
    api.verify_credentials()
    print("Authentication successful!")

except tweepy.TweepyException as e:
    print(f"Error during authentication: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Authentication successful!


In [4]:

import tweepy
import pandas as pd

# --- ---
client = tweepy.Client(bearer_token)

search_query = '(#Whoosh OR #KCIC OR #KeretaCepat OR "Whoosh" OR "KCIC" OR "kereta cepat") lang:id -is:retweet'


try:
    response = client.search_recent_tweets(
        query=search_query,
        max_results=100,
        tweet_fields=["created_at", "lang", "public_metrics"]
    )

    tweets = response.data

    if tweets:
        rows = []
        for tweet in tweets:
            rows.append({
                "tweet_id": tweet.id,
                "created_at": tweet.created_at,
                "lang": tweet.lang,
                "text": tweet.text,
                "retweet_count": tweet.public_metrics.get("retweet_count", 0),
                "reply_count": tweet.public_metrics.get("reply_count", 0),
                "like_count": tweet.public_metrics.get("like_count", 0),
                "quote_count": tweet.public_metrics.get("quote_count", 0),
            })

        df = pd.DataFrame(rows)
        print("Jumlah tweet:", len(df))
        print(df.head())
    else:
        print("Found 0 tweets.")
        df = pd.DataFrame()  # biar variabel df tetap ada

except tweepy.TweepyException as e:
    print(f"Error during tweet search: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Jumlah tweet: 100
              tweet_id                created_at lang  \
0  1990411250153537779 2025-11-17 13:26:36+00:00   in   
1  1990411152606540214 2025-11-17 13:26:13+00:00   in   
2  1990410892714934530 2025-11-17 13:25:11+00:00   in   
3  1990410336264954153 2025-11-17 13:22:58+00:00   in   
4  1990410250566963456 2025-11-17 13:22:38+00:00   in   

                                                text  retweet_count  \
0  Dirut KCIC soal Utang Whoosh: Kita Serahkan ke...              0   
1  @DS_yantie Tidak akan mau. Dia sadar butuh kon...              0   
2  @afannni @Strategi_Bisnis hhehe.... mestinya k...              0   
3  @ichannasution Selamat malam Kak. Mohon maaf m...              0   
4  @Galihholic_87 @Zeefire02 Eh termul brengsek k...              0   

   reply_count  like_count  quote_count  
0            1           0            0  
1            0           0            0  
2            0           0            0  
3            1           0            0  
4 